In [32]:
import PICclass as PIC
import numpy as np
import time

##################################
###   Name of the simulation   ###
##################################
prefix = "PIC_SHOCK_GAS_test4"

print("Studying simulation", prefix)
print("   ---------------------------------------------   ")

##################################
###    Path to get the data    ###
##################################
workdir = "/travail/wyao/data/" + prefix + "/PIC/"
##################################

##################################
###   Path to save the files   ###
##################################
savepath = "/travail/wyao/data/" + prefix + "/PIC_reduced/"

##################################
###       Simulation info      ###
##################################
nb_processor = 48
nb_particles = 0

#################################
###           Times           ###
#################################
PIC_Time_start = 0.0                                                          #Time when the PIC module was activated in the simulation; nanosecond unit
PIC_Time_end = 20.0 #29.307                                                          #Time when the PIC module wrote its last output files; nanosecond unit.
PIC_time_out_frequency = 2.0                                                   #Frequency for writing the output files; nanosecond unit.
l_restart = False                                                               #If you have already started to reorganize and the script crashed at some point for some reason
PIC_Time_restart = 12.0

#######################################
###   PICclass required arguments   ###
#######################################
Z = 1. #6. #1.                                                                 #Number of charge
atomic_weight = 1. #12. #2. 

('Studying simulation', 'PIC_SHOCK_GAS_test4')
   ---------------------------------------------   


In [3]:
###############################################################################
##           Function that rewrites all the original PIC files               ##
###############################################################################
def PICReorganizeOriginalFiles_In_limited(PIC_t_start, PIC_t_stop, PIC_tof, path):
    """Function that rewrites all of the PIC output files created by GORGON in order to make 1file/time for all the processors -
    the idea the reduction of the work that comes after by having only one file that can be loaded globaly using numpy.loadtxt -
    the header must be read by other means at the moment and there is still need to work on fast ways to distinguish the particles."""
#
    
    pic = PIC.PICreorganize(workdir, prefix, 0, nb_processor, path, Z, atomic_weight)
#     ipx,ipy,ipz,iPx,iPy,iPz,iBx,iBy,iBz,iEx,iEy,iEz,imass,icharge,itag,idx_nb_part,idx_PIC_dt,idx_PIC_time = pic.Header_OriginalFiles()
    ipx,ipy,ipz,iPx,iPy,iPz,iBx,iBy,iBz,iEx,iEy,iEz,imass,icharge,itag,iMHD_rho,iMHD_Ti,iMHD_Te,iMHD_Vx,iMHD_Vy,iMHD_Vz,idx_nb_part,idx_PIC_dt,idx_PIC_time = pic.Header_OriginalFiles()

    data_to_load = [ipx, ipy, ipz, iPx, iPy, iPz, iEx, iEy, iEz, iBx, iBy, iBz, imass, icharge, itag,  idx_nb_part, idx_PIC_dt]
    pic.SecurityCheckPaths_OriginalFiles("")
#
    iter_start = int(-1*(PIC_t_start/PIC_tof))
    iter_stop = int((PIC_t_stop-PIC_t_start)/PIC_tof)
    file_counter = 0
#
    for i in range(iter_start,iter_stop):
        if(i < 0):
            tmp_timing_start = time.time()
            """Creating a dummy empty file to fill the time gap between t=0ns and t=PIC_Time_start"""
            new_filename = "PIC_output" + str(file_counter) + ".dat"
            file_header = "#position_x position_y position_z impulsion_x impulsion_y impulsion_z E_x E_y E_z B_x B_y B_z mass charge tag " + str(0) + " " + str(0.0) + " " + str(file_counter*PIC_tof)

            Data = np.asarray([])
            np.savetxt(path+new_filename, Data, delimiter=' ', header=file_header)
#
            tmp_timing_end = time.time()
            print "Time: " + str(file_counter*PIC_tof) + " ns, dummy file: " + new_filename + " written | time taken: " + str(tmp_timing_end-tmp_timing_start)
            print " "
#
            file_counter += 1
        else :
            tmp_timing_start = time.time()
#
            pic = PIC.PICreorganize(workdir, prefix, i, nb_processor, path, Z, atomic_weight)
#
            nb_particles, list_procs = pic.CountParticles_OriginalFiles(idx_nb_part)
            Data = np.zeros([nb_particles,18])
#
            """Reading all the data for all the particles"""
            Data[:,1:] = pic.ExtractAllData_OriginalFiles(list_procs, data_to_load)
            """idx_nb_part and idx_PIC_dt are here used to load and save the quantities wrong_dt and nb_wrong_dt associated to given particle. """

            if np.size(Data[:,0]) != nb_particles :
                print("Warning 1 - Time: ", i, " problem: particles missing ", nb_particles - np.size(Data[:,0]))
#        
            """Once all of this is done you can save the global files """
            fname = workdir + "PIC_output" + str(0) + "." + str(i) + ".dat"
            dfile = open(fname, "r")
            f_header = dfile.readline().rstrip('\n\r').split()
            file_header = "#position_x position_y position_z impulsion_x impulsion_y impulsion_z E_x E_y E_z B_x B_y B_z mass charge tag " + str(nb_particles) + " " + str(f_header[idx_PIC_dt]) + " " + str(f_header[idx_PIC_time])

#
            """Saving the particles in a new global file"""
            new_filename = "PIC_output" + str(file_counter) + ".dat"
            np.savetxt(path+new_filename, Data, delimiter=' ', header=file_header)
#
            tmp_timing_end = time.time()
            print "Time: " + str(file_counter*PIC_tof), "ns, PIC file: ", new_filename, " written, nb of particles: ", str(nb_particles), " time taken: ", str(tmp_timing_end-tmp_timing_start)
            print " "
            file_counter += 1
#
    return


###############################################################################
##                                    End                                    ##
###############################################################################

In [33]:
print("Starting to rewrite the files - time:", time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()))
timing_start = time.time()

if not l_restart :
    PICReorganizeOriginalFiles_In_limited(PIC_Time_start, PIC_Time_end, PIC_time_out_frequency, savepath)
else :
    PICReorganizeOriginalFiles_Restart(PIC_Time_start, PIC_Time_restart, PIC_Time_end, PIC_time_out_frequency, savepath)

timing_end = time.time()
print("Ended to rewrite the files - time:", time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()))
print("Time taken:", (timing_end-timing_start)/60.)
print("End")

('Starting to rewrite the files - time:', '2020-09-15 21:41:13')


NameError: name 'PICReorganizeOriginalFiles_In_limited' is not defined

In [5]:
###############################################################################
##           Function that rewrites all the original PIC files               ##
###############################################################################
def PICReorganizeOriginalFiles_In(PIC_t_start, PIC_t_stop, PIC_tof, path):
    """Function that rewrites all of the PIC output files created by GORGON in order to make 1file/time for all the processors -
    the idea the reduction of the work that comes after by having only one file that can be loaded globaly using numpy.loadtxt -
    the header must be read by other means at the moment and there is still need to work on fast ways to distinguish the particles."""
#
    
    pic = PIC.PICreorganize(workdir, prefix, 0, nb_processor, path, Z, atomic_weight)
    ipx,ipy,ipz,iPx,iPy,iPz,iBx,iBy,iBz,iEx,iEy,iEz,imass,icharge,itag,iMHD_rho,iMHD_Ti,iMHD_Te,iMHD_Vx,iMHD_Vy,iMHD_Vz,idx_nb_part,idx_PIC_dt,idx_PIC_time = pic.Header_OriginalFiles()
    data_to_load = [ipx, ipy, ipz, iPx, iPy, iPz, iEx, iEy, iEz, iBx, iBy, iBz, imass, icharge, itag, iMHD_rho, iMHD_Ti, iMHD_Te, iMHD_Vx, iMHD_Vy, iMHD_Vz, idx_nb_part, idx_PIC_dt]
#     data_to_load = [ipx, ipy, ipz, iPx, iPy, iPz, iEx, iEy, iEz, iBx, iBy, iBz, imass, icharge, itag, iMHD_rho,                   iMHD_Vx, iMHD_Vy, iMHD_Vz, idx_nb_part, idx_PIC_dt] 
#     data_to_load = np.asarray([ipx, ipy, ipz, iPx, iPy, iPz, iBx, iBy, iBz, iEx, iEy, iEz, imass, icharge, itag, iMHD_rho, iMHD_Vx,iMHD_Vy,iMHD_Vz, idx_nb_part,idx_PIC_dt,idx_PIC_time])

    #   np.asarray([ipx, ipy, ipz, iPx, iPy, iPz, iBx, iBy, iBz, iEx, iEy, iEz, imass, icharge,       iMHD_rho, iMHD_Ti, iMHD_Te, iMHD_Vx, iMHD_Vy, iMHD_Vz, idx_wrong_dt, idx_nb_wrong_dt, itag]) #  [ipx, ipy, ipz, iPx, iPy, iPz, iBx, iBy, iBz, iEx, iEy, iEz, imass, icharge, idx_wrong_dt, idx_nb_wrong_dt, itag])
    pic.SecurityCheckPaths_OriginalFiles("")
#
    iter_start = int(-1*(PIC_t_start/PIC_tof))
    iter_stop = int((PIC_t_stop-PIC_t_start)/PIC_tof)
    file_counter = 0
#
    for i in range(iter_start,iter_stop):
        if(i < 0):
            tmp_timing_start = time.time()
            """Creating a dummy empty file to fill the time gap between t=0ns and t=PIC_Time_start"""
            new_filename = "PIC_output" + str(file_counter) + ".dat"
#             file_header = "#position_x position_y position_z impulsion_x impulsion_y impulsion_z E_x E_y E_z B_x B_y B_z mass charge tag MHD_density MHD_Ti MHD_Te MHD_Vx MHD_Vy MHD_Vz " + str(0) + " " + str(0.0) + " " + str(file_counter*PIC_tof)
            file_header = "#position_x position_y position_z impulsion_x impulsion_y impulsion_z E_x E_y E_z B_x B_y B_z mass charge tag MHD_density MHD_Vx MHD_Vy MHD_Vz " + str(0) + " " + str(0.0) + " " + str(file_counter*PIC_tof)

            Data = np.asarray([])
            np.savetxt(path+new_filename, Data, delimiter=' ', header=file_header)
#
            tmp_timing_end = time.time()
            print "Time: " + str(file_counter*PIC_tof) + " ns, dummy file: " + new_filename + " written | time taken: " + str(tmp_timing_end-tmp_timing_start)
            print " "
#
            file_counter += 1
        else :
            tmp_timing_start = time.time()
#
            pic = PIC.PICreorganize(workdir, prefix, i, nb_processor, path, Z, atomic_weight)
#
            nb_particles, list_procs = pic.CountParticles_OriginalFiles(idx_nb_part)
            Data = np.zeros([nb_particles,24])
#             Data = np.zeros([nb_particles,22])
#
            """Reading all the data for all the particles"""
            Data[:,1:] = pic.ExtractAllData_OriginalFiles(list_procs, data_to_load)
#             Data = pic.ExtractAllData_OriginalFiles(list_procs, data_to_load)
            """idx_nb_part and idx_PIC_dt are here used to load and save the quantities wrong_dt and nb_wrong_dt associated to given particle. """

            if np.size(Data[:,0]) != nb_particles :
                print("Warning 1 - Time: ", i, " problem: particles missing ", nb_particles - np.size(Data[:,0]))
#        
            """Once all of this is done you can save the global files """
            fname = workdir + "PIC_output" + str(0) + "." + str(i) + ".dat"
            dfile = open(fname, "r")
            f_header = dfile.readline().rstrip('\n\r').split()
            file_header = "#position_x position_y position_z impulsion_x impulsion_y impulsion_z E_x E_y E_z B_x B_y B_z mass charge tag MHD_density MHD_Ti MHD_Te MHD_Vx MHD_Vy MHD_Vz " + str(nb_particles) + " " + str(f_header[idx_PIC_dt]) + " " + str(f_header[idx_PIC_time])
#             file_header = "#position_x position_y position_z impulsion_x impulsion_y impulsion_z E_x E_y E_z B_x B_y B_z mass charge tag MHD_density MHD_Vx MHD_Vy MHD_Vz " + str(0) + " " + str(0.0) + " " + str(file_counter*PIC_tof)

#
            """Saving the particles in a new global file"""
            new_filename = "PIC_output" + str(file_counter) + ".dat"
            np.savetxt(path+new_filename, Data, delimiter=' ', header=file_header)
#
            tmp_timing_end = time.time()
            print "Time: " + str(file_counter*PIC_tof), "ns, PIC file: ", new_filename, " written, nb of particles: ", str(nb_particles), " time taken: ", str(tmp_timing_end-tmp_timing_start)
            print " "
            file_counter += 1
#
    return

def PICReorganizeOriginalFiles_Restart(PIC_t_start, PIC_t_restart, PIC_t_stop, PIC_tof, path):
    """Function that rewrites all of the PIC output files created by GORGON in order to make 1file/time for all the processors -
    the idea the reduction of the work that comes after by having only one file that can be loaded globaly using numpy.loadtxt -
    the header must be read by other means at the moment and there is still need to work on fast ways to distinguish the particles."""
#
    pic = PIC.PICreorganize(workdir, prefix, 0, nb_processor, path, Z, atomic_weight)
    ipx,ipy,ipz,iPx,iPy,iPz,iBx,iBy,iBz,iEx,iEy,iEz,imass,icharge,itag,iMHD_rho,iMHD_Ti,iMHD_Te,iMHD_Vx,iMHD_Vy,iMHD_Vz,idx_nb_part,idx_PIC_dt,idx_PIC_time = pic.Header_OriginalFiles()
    pic.SecurityCheckPaths_OriginalFiles("")
#
    iter_start = int((PIC_t_restart-PIC_t_start)/PIC_tof)
    iter_stop = iter_start + int((PIC_t_stop-PIC_t_restart)/PIC_tof)
    print PIC_t_start, PIC_t_restart, PIC_t_stop, iter_start, iter_stop
    file_counter = int(PIC_t_restart/PIC_tof)
#
    for i in range(iter_start, iter_stop):
        tmp_timing_start = time.time()
        print workdir + "PIC_output*." + str(i) + ".dat"
#
        pic = PIC.PICreorganize(workdir, prefix, i, nb_processor, path, Z, atomic_weight)
#
        nb_particles, list_procs = pic.CountParticles_OriginalFiles(idx_nb_part)
        Data = np.zeros([nb_particles,24])
#
        """Reading all the data for all the particles"""
        Data[:,1:] = pic.ExtractAllData_OriginalFiles(list_procs, [ipx, ipy, ipz, iPx, iPy, iPz, iEx, iEy, iEz, iBx, iBy, iBz, imass, icharge, itag, iMHD_rho, iMHD_Ti, iMHD_Te, iMHD_Vx, iMHD_Vy, iMHD_Vz, idx_nb_part, idx_PIC_dt])
        """idx_nb_part and idx_PIC_dt are here used to load and save the quantities wrong_dt and nb_wrong_dt associated to given particle. """
    
        if np.size(Data[:,0]) != nb_particles :
            print("Warning 1 - Time: ", i, " problem: particles missing ", nb_particles - np.size(Data[:,0]))
#        
        """Once all of this is done you can save the global files """
        fname = workdir + "PIC_output" + str(0) + "." + str(i) + ".dat"
        dfile = open(fname, "r")
        f_header = dfile.readline().rstrip('\n\r').split()
        file_header = "#position_x position_y position_z impulsion_x impulsion_y impulsion_z E_x E_y E_z B_x B_y B_z mass charge tag MHD_density MHD_Ti MHD_Te MHD_Vx MHD_Vy MHD_Vz " + str(nb_particles) + " " + str(f_header[idx_PIC_dt]) + " " + str(f_header[idx_PIC_time])
#
        """Saving the particles in a new global file"""
        new_filename = path + "PIC_output" + str(file_counter) + ".dat"
        np.savetxt(new_filename, Data, delimiter=' ', header=file_header)
#
        tmp_timing_end = time.time()
        print "Time: " + str(file_counter*PIC_tof), "ns, PIC file: ", new_filename, " written, nb of particles: ", str(nb_particles), " time taken: ", str(tmp_timing_end-tmp_timing_start)
        print " "
        file_counter += 1
#
    return

###############################################################################
##                                    End                                    ##
###############################################################################

In [26]:
print("Starting to rewrite the files - time:", time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()))
timing_start = time.time()

if not l_restart :
    PICReorganizeOriginalFiles_In(PIC_Time_start, PIC_Time_end, PIC_time_out_frequency, savepath)
else :
    PICReorganizeOriginalFiles_Restart(PIC_Time_start, PIC_Time_restart, PIC_Time_end, PIC_time_out_frequency, savepath)

timing_end = time.time()
print("Ended to rewrite the files - time:", time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()))
print("Time taken:", (timing_end-timing_start)/60., 'min')
print("End")

('Starting to rewrite the files - time:', '2020-09-15 09:16:40')
SecurityCheckPaths function: creation of the directory: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC_reduced/.
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output0.0.dat
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output1.0.dat
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output2.0.dat
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output3.0.dat
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output4.0.dat
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output5.0.dat
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output6.0.dat
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output7.0.dat
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output8.0.dat
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output9.0.dat
filename is: /travail/wyao/data/PIC_SHOCK_GAS_test2/PIC/PIC_output10.0.dat
filename is

In [13]:
###############################################################################
##           Function that rewrites all the original PIC files               ##
###############################################################################
def PICReorganizeOriginalFiles_In_limited(PIC_t_start, PIC_t_stop, PIC_tof, path):
    """Function that rewrites all of the PIC output files created by GORGON in order to make 1file/time for all the processors -
    the idea the reduction of the work that comes after by having only one file that can be loaded globaly using numpy.loadtxt -
    the header must be read by other means at the moment and there is still need to work on fast ways to distinguish the particles."""
#
    
    pic = PIC.PICreorganize(workdir, prefix, 0, nb_processor, path, Z, atomic_weight)
#     ipx,ipy,ipz,iPx,iPy,iPz,iBx,iBy,iBz,iEx,iEy,iEz,imass,icharge,itag,idx_nb_part,idx_PIC_dt,idx_PIC_time = pic.Header_OriginalFiles()
    ipx,ipy,ipz,iPx,iPy,iPz,iBx,iBy,iBz,iEx,iEy,iEz,imass,icharge,itag,iMHD_rho,iMHD_Ti,iMHD_Te,iMHD_Vx,iMHD_Vy,iMHD_Vz,idx_nb_part,idx_PIC_dt,idx_PIC_time = pic.Header_OriginalFiles()

    data_to_load = [ipx, ipy, ipz, iPx, iPy, iPz, iEx, iEy, iEz, iBx, iBy, iBz, imass, icharge, itag,  idx_nb_part, idx_PIC_dt]
    pic.SecurityCheckPaths_OriginalFiles("")
#
    iter_start = int(-1*(PIC_t_start/PIC_tof))
    iter_stop = int((PIC_t_stop-PIC_t_start)/PIC_tof)
    file_counter = 0
#
    for i in range(iter_start,iter_stop):
        if(i < 0):
            tmp_timing_start = time.time()
            """Creating a dummy empty file to fill the time gap between t=0ns and t=PIC_Time_start"""
            new_filename = "PIC_output" + str(file_counter) + ".dat"
            file_header = "#position_x position_y position_z impulsion_x impulsion_y impulsion_z E_x E_y E_z B_x B_y B_z mass charge tag " + str(0) + " " + str(0.0) + " " + str(file_counter*PIC_tof)

            Data = np.asarray([])
            np.savetxt(path+new_filename, Data, delimiter=' ', header=file_header)
#
            tmp_timing_end = time.time()
            print "Time: " + str(file_counter*PIC_tof) + " ns, dummy file: " + new_filename + " written | time taken: " + str(tmp_timing_end-tmp_timing_start)
            print " "
#
            file_counter += 1
        else :
            tmp_timing_start = time.time()
#
            pic = PIC.PICreorganize(workdir, prefix, i, nb_processor, path, Z, atomic_weight)
#
            nb_particles, list_procs = pic.CountParticles_OriginalFiles(idx_nb_part)
            Data = np.zeros([nb_particles,18])
#
            """Reading all the data for all the particles"""
            Data[:,1:] = pic.ExtractAllData_OriginalFiles(list_procs, data_to_load)
            """idx_nb_part and idx_PIC_dt are here used to load and save the quantities wrong_dt and nb_wrong_dt associated to given particle. """

            if np.size(Data[:,0]) != nb_particles :
                print("Warning 1 - Time: ", i, " problem: particles missing ", nb_particles - np.size(Data[:,0]))
#        
            """Once all of this is done you can save the global files """
            fname = workdir + "PIC_output" + str(0) + "." + str(i) + ".dat"
            dfile = open(fname, "r")
            f_header = dfile.readline().rstrip('\n\r').split()
            file_header = "#position_x position_y position_z impulsion_x impulsion_y impulsion_z E_x E_y E_z B_x B_y B_z mass charge tag" + str(nb_particles) + " " + str(f_header[idx_PIC_dt]) + " " + str(f_header[idx_PIC_time])

#
            """Saving the particles in a new global file"""
            new_filename = "PIC_output" + str(file_counter) + ".dat"
            np.savetxt(path+new_filename, Data, delimiter=' ', header=file_header)
#
            tmp_timing_end = time.time()
            print "Time: " + str(file_counter*PIC_tof), "ns, PIC file: ", new_filename, " written, nb of particles: ", str(nb_particles), " time taken: ", str(tmp_timing_end-tmp_timing_start)
            print " "
            file_counter += 1
#
    return


###############################################################################
##                                    End                                    ##
###############################################################################

In [5]:
4.6655e+03 * 600000 / 50000 / 3600

15.551666666666666

In [8]:
4.3934e+01 / 60

0.7322333333333333

In [30]:
data_to_load = [ipx, ipy, ipz, 
                    iPx, iPy, iPz, 
                    iEx, iEy, iEz, 
                    iBx, iBy, iBz, 
                    imass, icharge, itag, 
                    iMHD_rho, iMHD_Ti, iMHD_Te, 
                    iMHD_Vx, iMHD_Vy, iMHD_Vz, 
                    idx_nb_part, idx_PIC_dt, idx_PIC_time]

In [29]:
pic = PIC.PICreorganize(workdir, prefix, 0, nb_processor, savepath, Z, atomic_weight)
ipx, ipy, ipz, \
iPx, iPy, iPz, \
iBx, iBy, iBz, \
iEx, iEy, iEz, \
imass, icharge, itag, \
iMHD_rho, iMHD_Ti, iMHD_Te, \
iMHD_Vx, iMHD_Vy, iMHD_Vz, \
idx_nb_part, idx_PIC_dt, idx_PIC_time = pic.Header_OriginalFiles()

In [31]:
np.size(data_to_load)

24